Exercice 1 : Calculer la distance entre deux points :
* Écrivez une fonction qui calcule la distance entre deux valeurs.
Cela servira de base pour chaque paire de points dans les deux séries temporelles.


In [7]:
def distance(a, b):
    return abs(a - b)

Exercice 2 : Matrice de coût cumulative :
* Créer une matrice de coût cumulative où chaque cellule représente le coût minimum pour atteindre ce point en suivant les règles de DTW.


# Consider i as j and j as i 

In [49]:
import numpy as np

def calcul_matrice_dtw(serie1, serie2):
    n, m = len(serie1), len(serie2)
    matrice_dtw = np.zeros((n, m))
    
    #Initialiser la première cellule
    matrice_dtw[0, 0] = distance(serie1[0], serie2[0])
    matrice_dtw[0,1:] = -1
    # dpp (1,1) = d(1,1)
    # dpp(1,j) = infinity j > 1
    # dpp(i,j) = min {dpp(i-1,j) + d(i,j)
    #                 dpp(i-1,j-1) + 2*d(i,j)
    #                 dpp(i,j-1) + d(i,j) }
    for k in range(1, n):
        matrice_dtw[k,0] = matrice_dtw[k-1,0] + distance(serie1[k],serie2[0])
        
    for i in range(1,n):
        for j in range(1,m):
            dist = distance(serie1[i], serie2[j])
            if matrice_dtw[i, j-1] != -1 and matrice_dtw[i-1, j-1] != -1 and matrice_dtw[i-1, j] != -1:
                dpp1 = matrice_dtw[i, j-1] + dist
                dpp2 = matrice_dtw[i-1, j-1] + 2 * dist
                dpp3 = matrice_dtw[i-1, j] + dist
                matrice_dtw[i,j] = min(dpp1,dpp2,dpp3)
                
            elif matrice_dtw[i, j-1] != -1 and matrice_dtw[i-1, j-1] != -1:
                dpp1 = matrice_dtw[i, j-1] + dist
                dpp2 = matrice_dtw[i-1, j-1] + 2 * dist
                matrice_dtw[i,j] = min(dpp1,dpp2)
            
            elif matrice_dtw[i, j-1] != -1:
                dpp1 = matrice_dtw[i, j-1] + dist
                matrice_dtw[i,j] = dpp1
                
            elif matrice_dtw[i-1, j-1] != -1:
                dpp2 = matrice_dtw[i-1, j-1] + 2 * dist
                matrice_dtw[i,j] = dpp2
                
            elif matrice_dtw[i-1, j] != -1:
                dpp3 = matrice_dtw[i-1, j] + dist
                matrice_dtw[i,j] = dpp3

    return matrice_dtw

Exercice 3 : Chemin d'alignement optimal :
* Une fois la matrice remplie, Créer une fonction pour retracer le chemin d'alignement optimal en partant du coin supérieur gauche jusqu'au coin  inferieur droit.


In [9]:
def chemin_optimal(matrice_dtw):
    chemin = 0
    return chemin

In [51]:
def retracer_chemin(matrice):
    # Initialisation des coordonnées de départ
    i, j = len(matrice) - 1, len(matrice[0]) - 1
    chemin = []

    # On remonte dans la matrice jusqu'à atteindre le coin supérieur gauche
    while i > 0 or j > 0:
        chemin.append((i, j))  # On ajoute la position actuelle au chemin

        # Vérification des directions possibles
        if i > 0 and j > 0:  # On peut aller en diagonale
            if matrice[i][j] == matrice[i-1][j-1] + 1:
                i -= 1
                j -= 1
            elif matrice[i][j] == matrice[i-1][j] -1:
                i -= 1
            elif matrice[i][j] == matrice[i][j-1] -1:
                j -= 1
        elif i > 0:  # On est sur la première colonne
            i -= 1
        elif j > 0:  # On est sur la première ligne
            j -= 1

    # Ajouter le coin supérieur gauche
    chemin.append((0, 0))
    
    # Inverser le chemin pour avoir l'ordre correct (du haut-gauche au bas-droit)
    chemin.reverse()
    return chemin


Exercice 4 : Visualisation :
* visualiser les séries et le chemin d’alignement en utilisant Matplotlib.
Cela vous aidera à voir comment les points de deux séries sont alignés.

In [ ]:
import matplotlib.pyplot as plt
def visualiser_alignement(serie1, serie2, chemin):
    plt.figure(figsize=(10, 6))
    plt.plot(range(n), [0] * n, 'ro-', label='Série 1 (s1)', markersize=10)  # Série 1
    plt.plot(range(m), [1] * m, 'bo-', label='Série 2 (s2)', markersize=10)  # Série 2

# Tracer le chemin d'alignement
    for (i, j) in chemin_optimal:
        plt.plot([i, j], [0, 1], 'k--')  # Lignes pointillées entre les points

# Annotations
    plt.xticks(range(max(n, m)), [s1[i] if i < n else '' for i in range(max(n, m))])
    plt.yticks([0, 1], ['Série 1', 'Série 2'])
    plt.title("Visualisation de l'Alignement Optimal")
    plt.xlabel("Position dans la Série 1")
    plt.ylabel("Position dans la Série 2")
    plt.legend()
    plt.grid()
    plt.show()

Exercice final : Comparaison de séries :
* Voici deux séries temporelles, calculer la distance DTW, de trouver le chemin d’alignement optimal, et de visualiser les résultats.


In [ ]:
serie1 = [1, 2, 3, 4, 5]
serie2 = [2, 3, 4, 5, 6, 7]

matrice = calcul_matrice_dtw(serie1, serie2)
print(f"La matrice: \n{matrice}")

chemin = chemin_optimal(matrice)
print(f"Chemin: \n{chemin}")
visualiser_alignement(serie1, serie2, chemin)

La matrice: 
[[ 1. -1. -1. -1. -1. -1.]
 [ 1.  2.  4.  7. 11. 16.]
 [ 2.  1.  2.  4.  7. 11.]
 [ 4.  2.  1.  2.  4.  7.]
 [ 7.  4.  2.  1.  2.  4.]]
chemin
0
